# Validation - Star positions + kinematics

Generates the plots presented in SynthPop Paper 1, Section 6.2

Macy Huston & Jonas Klüter

In [ ]:
import sys
import os

import numpy as np
import synthpop
import matplotlib.pyplot as plt
import pandas as pd

### Generate a few test catalogs

In [ ]:
# Set up model object 
model = synthpop.SynthPop(model_name='besancon_Robin2003',name_for_output='Besancon_validation',obsmag=False, 
                        maglim=['2MASS_Ks', 999999, 'keep'], extinction_map_kwargs={'name':'NoExtinction'}, 
                        l_set=[0,90, 12.17, 39.14], b_set=[0, 0, 5.37, 8.53], l_set_type="pairs", b_set_type="pairs",
                        solid_angle=[1e-3,3e-1,6e-2, 6e-1], solid_angle_unit='deg^2',
                        max_distance=25, overwrite=True, output_file_type='hdf5')
model.init_populations()

In [ ]:
# Only re-run if needed - catalog generation
if True:
    for i,loc in enumerate(model.get_iter_loc()):
        # run synthpop for the given location and solid angle
        model.process_location(*loc, model.parms.solid_angle[i], model.parms.solid_angle_unit,save_data=True)

In [ ]:
# Catalog load
data0 = pd.read_hdf("/u/mhuston/code/synthpop/synthpop/outputfiles/Besancon_validation/besancon_Robin2003_l0.000_b0.000.h5")
data1 = pd.read_hdf("/u/mhuston/code/synthpop/synthpop/outputfiles/Besancon_validation/besancon_Robin2003_l90.000_b0.000.h5")
data2 = pd.read_hdf("/u/mhuston/code/synthpop/synthpop/outputfiles/Besancon_validation/besancon_Robin2003_l12.170_b5.370.h5")
data3 = pd.read_hdf("/u/mhuston/code/synthpop/synthpop/outputfiles/Besancon_validation/besancon_Robin2003_l39.140_b8.530.h5")
catalogs = [data0,data1,data2,data3]

## Let's look at stellar density

In [ ]:
# Set up histogram bins, and points for density function plotting.
dist_bins = np.arange(0.0,25.01, 0.5)
dist_binc = dist_bins[:-1] + np.diff(dist_bins)/2
dist_func_pts = np.arange(0.0,25.0001, 0.05)
ccycle = ['#377eb8', '#ff7f00', '#4daf4a', '#f781bf', '#a65628', '#984ea3', '#999999', '#e41a1c', '#dede00']

In [ ]:
# Generic plotting code we can use for any population
def plot_pop_dens(pops, poplabel=None, ylim=None, legend=True):
    # Cycle through sight lines
    for sl in range(len(model.parms.l_set)):
        sum_catalog = np.zeros(len(dist_binc))
        sum_func = np.zeros(len(dist_func_pts))
        # Cycle through populations
        for pop in pops:
            # Bin catalog data - sum mass and divide by volume
            data_sl = catalogs[sl]
            l,b = model.parms.l_set[sl], model.parms.b_set[sl]
            cone_rads = np.sqrt(model.parms.solid_angle[sl]/np.pi) *np.pi/180
            cone_vol = np.pi * (cone_rads*dist_bins)**2 * dist_bins / 3
            cone_chunks = np.diff(cone_vol)
            binned_dists = pd.cut(data_sl.Dist, dist_bins)
            if model.populations[pop].population_density.density_unit=='number':
                sum_catalog += np.histogram(data_sl.where(data_sl['pop']==pop).Dist, bins=dist_bins)[0]
            else:
                sum_catalog += data_sl.where(data_sl['pop']==pop).groupby(binned_dists).Mass.sum()
            # Get density function from model object
            r,phi,z = model.populations[pop].coord_trans.dlb_to_rphiz(dist_func_pts, l,b)
            sum_func += model.populations[pop].population_density.density(r,phi,z)
        sl_lab = '('+str(l)+', '+str(b)+')'
        # Plot the summed data
        plt.step(dist_binc, sum_catalog/cone_chunks, where='mid', c=ccycle[sl])
        plt.plot(dist_func_pts, sum_func, 
                     c=ccycle[sl], linewidth=5, alpha=0.25, label=sl_lab)
    # Plot labeling/formatting
    plt.yscale('log')
    if model.populations[pop].population_density.density_unit=='number':
        plt.ylabel(r'Stellar Density [*/kpc$^3$]')
    else:
        plt.ylabel(r'Stellar Density [M$_\odot$/kpc$^3$]')
    plt.xlabel('Distance [kpc]')
    plt.xlim(0,25); plt.ylim(ylim)
    if legend: plt.legend()
    plt.title(poplabel, loc='left')
    plt.tight_layout()
    plt.savefig('validation_figures/'+poplabel.replace(' ','_')+'.pdf')

In [ ]:
plot_pop_dens([0], poplabel='bulge', ylim=(1e4,3e10), legend=True)

In [ ]:
plot_pop_dens([1],poplabel='halo',ylim=(1e3,2e7), legend=False)

In [ ]:
plot_pop_dens([2], ylim=(1e4,1e8), poplabel='thick disk', legend=False)

In [ ]:
plot_pop_dens([3,4,5,6,7,8,9], ylim=(1e4,5e8), poplabel='thin disk', legend=False)

## Next: kinematics

In [ ]:
kine_func_pts = np.arange(0.0,22.1501, 0.05)

In [ ]:
def plot_kinematics_distr(sl, pop,ds=1):
    l,b = model.parms.l_set[sl], model.parms.b_set[sl]
    data = catalogs[sl][::ds]
    x,y,z = model.populations[pop].coord_trans.dlb_to_xyz(kine_func_pts, l,b)
    r,phi_rad,z2 = model.populations[pop].coord_trans.dlb_to_rphiz(kine_func_pts, l,b)
    
    u1,v1,w1 = model.populations[pop].kinematics.mean_galactic_uvw(x,y,z)
    u1p = u1 + model.populations[pop].kinematics.sigma_u * np.exp((r-model.populations[pop].kinematics.sun.r)*model.populations[pop].kinematics.disp_grad/2)
    v1p,w1p = v1 + model.populations[pop].kinematics.sigma_v, w1 + model.populations[pop].kinematics.sigma_w
    u1m = u1 - model.populations[pop].kinematics.sigma_u * np.exp((r-model.populations[pop].kinematics.sun.r)*model.populations[pop].kinematics.disp_grad/2)
    v1m,w1m = v1 - model.populations[pop].kinematics.sigma_v, w1 - model.populations[pop].kinematics.sigma_w
    
    u,v,w = u1 * np.cos(phi_rad) + v1 * np.sin(phi_rad), -u1 * np.sin(phi_rad) + v1 * np.cos(phi_rad), w1
    u_p,v_p,w_p = u1p * np.cos(phi_rad) + v1p * np.sin(phi_rad), -u1p * np.sin(phi_rad) + v1p * np.cos(phi_rad), w1p
    u_m,v_m,w_m = u1m * np.cos(phi_rad) + v1m * np.sin(phi_rad), -u1m * np.sin(phi_rad) + v1m * np.cos(phi_rad), w1m
    
    plt.subplots(nrows=1,ncols=3,figsize=(15,5))
    plt.subplot(131)
    plt.plot(data.where(data['pop']==pop).Dist, np.abs(data.where(data['pop']==pop)['V']), marker='.', markersize=0.2, linestyle='none', c='k')
    plt.plot(kine_func_pts, np.abs(v), c='c', label='Function mean')
    plt.plot(kine_func_pts, np.abs(v_p), c='c', linestyle='--', label=r'Function $\pm$ 1-sigma')
    plt.plot(kine_func_pts, np.abs(v_m), c='c', linestyle='--')
    plt.ylabel('|V| (km/s)'); plt.xlabel('Distance (kpc)')
    plt.ylim(100,400)
    plt.xlim(0,25)
    
    plt.subplot(132)
    plt.plot(data.where(data['pop']==pop).Dist, data.where(data['pop']==pop)['U'], marker='.', markersize=0.2, linestyle='none', c='k')
    plt.plot(kine_func_pts, u, c='c', label='Function mean')
    plt.plot(kine_func_pts, u_p, c='c', linestyle='--', label=r'Function $\pm$ 1-sigma')
    plt.plot(kine_func_pts, u_m, c='c', linestyle='--')
    plt.ylabel('U (km/s)'); plt.xlabel('Distance (kpc)')
    plt.ylim(-250,250)
    plt.xlim(0,25)
    
    plt.subplot(133)
    plt.plot(data.where(data['pop']==pop).Dist, data.where(data['pop']==pop)['W'], marker='.', markersize=0.2, linestyle='none', c='k')
    plt.plot(kine_func_pts, w, c='c', label='Function mean')
    plt.plot(kine_func_pts, w_p, c='c', linestyle='--', label=r'Function $\pm$ 1$\sigma$')
    plt.plot(kine_func_pts, w_m, c='c', linestyle='--')
    plt.ylabel('W (km/s)'); plt.xlabel('Distance (kpc)')
    plt.legend()
    plt.ylim(-110,110)
    plt.xlim(0,25)
    
    plt.tight_layout()
    plt.savefig('validation_figures/kinematics_sl'+str(sl)+'_pop'+str(pop)+'_ds'+str(ds)+'.pdf')

In [ ]:
plot_kinematics_distr(0,9,ds=2)